*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [45]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.9.0-dev20220312


In [46]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-03-12 23:27:04--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2022-03-12 23:27:04 (8.56 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2022-03-12 23:27:04--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [47]:
df_train = pd.read_csv(train_file_path,sep="\t",header=None)

In [48]:
df_train = df_train.rename(columns={0: 'HamOrSpam',1: 'Text'})
df_train

,HamOrSpam,Text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [49]:
df_test = pd.read_csv(test_file_path,sep="\t",header=None)
df_test

,0,1
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [50]:
df_test = df_test.rename(columns={0: 'HamOrSpam',1: 'Text'})
df_test

,HamOrSpam,Text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [51]:
# Convert Categorical to numeric.
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
df_train["HamOrSpam"] = labelEncoder.fit_transform(df_train["HamOrSpam"])
df_test["HamOrSpam"] = labelEncoder.fit_transform(df_test["HamOrSpam"])

In [52]:
df_train.head()

,HamOrSpam,Text
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [53]:
df_test.head()

,HamOrSpam,Text
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?


In [54]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4179 entries, 0 to 4178
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HamOrSpam  4179 non-null   int64 
 1   Text       4179 non-null   object
dtypes: int64(1), object(1)
memory usage: 65.4+ KB


In [55]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392 entries, 0 to 1391
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HamOrSpam  1392 non-null   int64 
 1   Text       1392 non-null   object
dtypes: int64(1), object(1)
memory usage: 21.9+ KB


In [56]:
df_train["HamOrSpam"].value_counts(normalize=True)

0    0.865997
1    0.134003
Name: HamOrSpam, dtype: float64

In [57]:
X_train = df_train["Text"]
X_test = df_test["Text"]
y_train = df_train["HamOrSpam"]
y_test = df_test["HamOrSpam"]

In [58]:
print(X_train.head())
print(X_test.head())
print(y_train.head())
print(y_test.head())

0    ahhhh...just woken up!had a bad dream about u ...
1                             you can never do nothing
2    now u sound like manky scouse boy steve,like! ...
3    mum say we wan to go then go... then she can s...
4    never y lei... i v lazy... got wat? dat day ü ...
Name: Text, dtype: object
0    i am in hospital da. . i will return home in e...
1           not much, just some textin'. how bout you?
2    i probably won't eat at all today. i think i'm...
3    don‘t give a flying monkeys wot they think and...
4                                  who are you seeing?
Name: Text, dtype: object
0    0
1    0
2    0
3    0
4    0
Name: HamOrSpam, dtype: int64
0    0
1    0
2    0
3    0
4    0
Name: HamOrSpam, dtype: int64


In [59]:
X_train = np.array(X_train)

In [60]:
X_test = np.array(X_test)

In [61]:
vectorize_layer = keras.layers.TextVectorization()
vectorize_layer.adapt(X_train)


In [62]:
vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'to',
 'i',
 'you',
 'a',
 'the',
 'u',
 'and',
 'in',
 'is',
 'me',
 'my',
 'for',
 'your',
 'of',
 'it',
 'call',
 'have',
 'on',
 'that',
 'are',
 'now',
 'im',
 '2',
 'not',
 'but',
 'so',
 'at',
 'or',
 'do',
 'can',
 'with',
 'be',
 'will',
 'if',
 'get',
 'ur',
 'just',
 'we',
 'this',
 'no',
 'its',
 'up',
 'go',
 '4',
 'ok',
 'dont',
 'ltgt',
 'free',
 'when',
 'out',
 'how',
 'all',
 'what',
 'from',
 'know',
 'like',
 'got',
 'then',
 'come',
 'good',
 'ill',
 'time',
 'am',
 'was',
 'only',
 'day',
 'he',
 'love',
 'send',
 'there',
 'as',
 'want',
 'text',
 'going',
 'by',
 'ü',
 'one',
 'about',
 'need',
 'txt',
 'lor',
 'still',
 'our',
 'see',
 'n',
 'r',
 'home',
 'stop',
 'sorry',
 'reply',
 'back',
 'today',
 'tell',
 'she',
 'new',
 'been',
 'mobile',
 'later',
 'take',
 'hi',
 'da',
 'her',
 'they',
 'well',
 'think',
 'night',
 'has',
 'any',
 'please',
 'much',
 'cant',
 'week',
 'dear',
 'an',
 'happy',
 'some',
 'pls',
 'hey',
 'did',
 'phone',


In [63]:
vocab_size = len(vectorize_layer.get_vocabulary())

In [64]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=32,
        mask_zero=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [65]:
model.compile(optimizer="adam",
             loss = "binary_crossentropy",
             metrics=['accuracy'])


In [66]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, None, 32)          261792    
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 271,201
Trainable params: 271,201
Non-trainable params: 0
________________________________________________

In [67]:
# early stopping callback
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                  mode="min", 
                                  patience=5,
                                  restore_best_weights=True)

In [68]:
mdl_history = model.fit(X_train,
         y_train,
         epochs = 15,
         validation_data = (X_test,y_test),
         callbacks = [callback])

Epoch 1/15
131/131 [==============================] - 22s 98ms/step - loss: 0.3014 - accuracy: 0.9189 - val_loss: 0.0702 - val_accuracy: 0.9784
Epoch 2/15
131/131 [==============================] - 5s 42ms/step - loss: 0.0399 - accuracy: 0.9907 - val_loss: 0.0402 - val_accuracy: 0.9878
Epoch 3/15
131/131 [==============================] - 6s 44ms/step - loss: 0.0147 - accuracy: 0.9974 - val_loss: 0.0446 - val_accuracy: 0.9892
Epoch 4/15
131/131 [==============================] - 5s 41ms/step - loss: 0.0049 - accuracy: 0.9993 - val_loss: 0.0592 - val_accuracy: 0.9813
Epoch 5/15
131/131 [==============================] - 5s 40ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0444 - val_accuracy: 0.9885
Epoch 6/15
131/131 [==============================] - 5s 42ms/step - loss: 8.5435e-04 - accuracy: 1.0000 - val_loss: 0.0485 - val_accuracy: 0.9892
Epoch 7/15
131/131 [==============================] - 5s 42ms/step - loss: 6.5203e-04 - accuracy: 1.0000 - val_loss: 0.0510 - val_accurac

In [69]:
pred_text = "sale today! to stop texts call 98912460324"

In [70]:
np.array([pred_text])

array(['sale today! to stop texts call 98912460324'], dtype='<U42')

In [71]:
result = model.predict(np.array([pred_text]))

1/1 [==============================] - 2s 2s/step


In [72]:
result[0][0]

0.9189689

In [73]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  result = model.predict(np.array([pred_text]))
  if (result[0][0] >= 0.5):
    hamOrSpam = 'spam'
  else:
    hamOrSpam = 'ham'
  prediction = [result[0][0],hamOrSpam]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 22ms/step
[0.00405377, 'ham']


In [74]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:  
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 25ms/step
You passed the challenge. Great job!
